# SQLite integration walkthrough

This notebook explains the SQLite-related code in `include/Database.hpp`, `src/Database.cpp`, and `src/main.cpp` line by line.

## `include/Database.hpp`
- `#pragma once` prevents multiple inclusions of the header.
- `#include <sqlite3.h>` pulls the C SQLite API; the rest (`<stdexcept>`, `<string>`) are for exceptions and string handling.
- `class Statement` wraps a prepared statement `sqlite3_stmt*` to give RAII and typed helpers:
  - Constructor takes a `sqlite3* db` and SQL string; it calls `sqlite3_prepare_v2` (definition in the .cpp) and throws on failure.
  - Destructor finalizes the statement (`sqlite3_finalize`) so resources are freed automatically.
  - Copy/move operations are deleted to avoid double-finalizing the same `sqlite3_stmt*`.
  - `bindText`, `bindDouble`, `bindInt` bind parameters at 1-based positions.
  - `step()` runs `sqlite3_step`: returns `true` if a row is produced, `false` if the statement is done; throws on error. `done()` reports whether `SQLITE_DONE` was seen.
  - `columnText/Int/Double` read result columns of the current row.
  - Private fields: `sqlite3_stmt* stmt_{nullptr}` stores the native handle; `bool done_{false}` tracks completion.
- `class Database` wraps an `sqlite3*` connection:
  - Constructor takes a file path and opens the DB; destructor closes it.
  - Copy/move are deleted to avoid multiple owners of the same connection.
  - `handle()` exposes the raw `sqlite3*` for use by helper functions.
  - `exec` runs a raw SQL string (no bound params) and throws on error.
  - `initializeSchema` creates all tables/PRAGMAs (implemented in the .cpp).

## `src/Database.cpp`
- Includes its own header and `<utility>` (though not used here).

### Statement
- Constructor: `sqlite3_prepare_v2(db, sql.c_str(), -1, &stmt_, nullptr)` compiles the SQL text into a prepared statement; on non-`SQLITE_OK` it throws with `sqlite3_errmsg(db)`.
- Destructor: calls `sqlite3_finalize(stmt_)` if non-null to release the prepared statement.
- `bindText`: binds UTF-8 text at the given index; `-1` tells SQLite to compute string length; `SQLITE_TRANSIENT` copies the data. Throws on non-`SQLITE_OK`.
- `bindDouble` / `bindInt`: bind numeric parameters; throw on error.
- `step`: runs `sqlite3_step`. If it returns `SQLITE_ROW`, there is a result row (return `true`). If `SQLITE_DONE`, the statement is finished (set `done_ = true`, return `false`). Anything else is an error -> throw.
- `done`: accessor for the `done_` flag.
- `columnText/Int/Double`: read column values from the current row via `sqlite3_column_*`; `columnText` guards against `nullptr` by returning an empty string when no text is present.

### Database
- Constructor: `sqlite3_open(path.c_str(), &db_)` opens (or creates) the database file. On failure, it throws including `sqlite3_errmsg(db_)`.
- Destructor: closes the connection with `sqlite3_close` if it was opened.
- `exec`: runs SQL without bound parameters using `sqlite3_exec`. If it fails, grabs the error string, frees it with `sqlite3_free`, and throws.
- `initializeSchema`: sets up the schema.
  - `PRAGMA foreign_keys = ON;` enforces foreign-key constraints.
  - `members` table: `id` PK, `name` required, optional `cpf`, `phone`, `pix`.
  - `payments` table: rows tied to a `member_id`, with `amount`, `date`, `duration`; `ON DELETE CASCADE` removes payments when a member is deleted.
  - `checkins` table: check-in records per member; cascading delete.
  - `workout_plans` table: one plan per member (as modeled here); cascades on member delete.
  - `workout_plan_days` table: days within a plan, keyed by `plan_id` and a `day_index` (0-based per current code); cascades on plan delete.
  - `exercises` table: exercises per day with name, sets, reps, weight; cascades on day delete.
  Each `exec` call runs a `CREATE TABLE IF NOT EXISTS ...` block, so rerunning is idempotent.

## `src/main.cpp`
- Includes iostream/vector plus project headers: `CheckIn`, `Database`, `Exercise`, `Member`, `Payment`, `WorkoutPlan`.

### Helper functions
- `insertMember`: prepares an `INSERT` with 4 placeholders, binds name/cpf/phone/pix, calls `step()` to execute (no result rows), and returns the new row id via `sqlite3_last_insert_rowid`.
- `insertPayment`: prepared insert into `payments`, binding `member_id`, `amount` (double), `date`, `duration`; executes with `step()`.
- `insertCheckIn`: similar insert into `checkins`, binding member, date, time.
- `insertWorkoutPlan`: inserts a plan row for the member, grabs `planId` via `last_insert_rowid`, then iterates the `WorkoutPlan` days. For each day, it inserts into `workout_plan_days`, captures `dayId`, and inserts each `Exercise` into `exercises` with bound name/sets/reps/weight. Returns the plan id.

### `main`
- Wraps everything in a `try/catch` to print errors and return 1 on failure.
- Creates a `Database db("gym.db")` connection to a local file and calls `initializeSchema()` to ensure tables exist.
- Builds a sample `WorkoutPlan` with two days and sample `Exercise` entries.
- Builds a `Member` with name/cpf/phone/pix, one `Payment`, one `CheckIn`, and the plan.
- Persists the member and related aggregates:
  - `insertMember` yields `memberId`.
  - Loops through `member.payments()` and `member.checkins()` inserting each with the member id.
  - Calls `insertWorkoutPlan` to persist the plan, its days, and exercises.
- Runs four simple `SELECT COUNT(*)` queries using `Statement` to show row counts for members, payments, checkins, and exercises, printing to stdout.
- On success returns 0; on exception prints the error and returns 1.